<a href="https://colab.research.google.com/github/mostafa-ja/mal_adv4/blob/main/RBF2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

download_links = ['https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_0.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_1.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_2.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y0.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y1.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y2.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/model_DNN_drebin_best.pth',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/model_AT_rFGSM_weightedLoss.pth',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/model_AT_rFGSM.pth',
                  'https://github.com/mostafa-ja/mal_adv4/raw/main/dataset/insertion_array.pkl',
                  'https://github.com/mostafa-ja/mal_adv4/raw/main/dataset/removal_array.pkl',
                  'https://github.com/mostafa-ja/mal_adv4/raw/main/dataset/adverserial_attacks_functions.py'
]

In [4]:
import gdown
output_filepath = '/content/'
for link in download_links:
  gdown.download(link, output_filepath)


Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_0.npz
To: /content/sparse_matrix_0.npz
100%|██████████| 461k/461k [00:00<00:00, 12.5MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_1.npz
To: /content/sparse_matrix_1.npz
100%|██████████| 148k/148k [00:00<00:00, 5.89MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_2.npz
To: /content/sparse_matrix_2.npz
100%|██████████| 150k/150k [00:00<00:00, 5.32MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y0.npz
To: /content/sparse_matrix_y0.npz
100%|██████████| 5.79k/5.79k [00:00<00:00, 11.9MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y1.npz
To: /content/sparse_matrix_y1.npz
100%|██████████| 2.64k/2.64k [00:00<00:00, 7.50MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y2

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,balanced_accuracy_score
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import pickle
import random
import time

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

from adverserial_attacks_functions import *


torch.manual_seed(0)



cpu
cpu


In [6]:
# Open the .pkl file
with open('/content/insertion_array.pkl', 'rb') as f:
    # Load the object
    insertion_array = pickle.load(f)

# Close the file
f.close()

insertion_array = torch.tensor(insertion_array).to(device)
print(len(insertion_array))

# Open the .pkl file
with open('/content/removal_array.pkl', 'rb') as f:
    # Load the object
    removal_array = pickle.load(f)

# Close the file
f.close()

removal_array = torch.tensor(removal_array).to(device)
print(len(removal_array))


10000
10000


In [7]:

# Load dataset
X_train = sparse.load_npz("/content/sparse_matrix_0.npz").toarray()
X_val = sparse.load_npz("/content/sparse_matrix_1.npz").toarray()
X_test = sparse.load_npz("/content/sparse_matrix_2.npz").toarray()

X_train = torch.tensor(X_train, dtype=torch.int8)
X_val = torch.tensor(X_val, dtype=torch.int8)
X_test = torch.tensor(X_test, dtype=torch.int8)


y_train = sparse.load_npz("/content/sparse_matrix_y0.npz").toarray().reshape((-1, 1))
y_val = sparse.load_npz("/content/sparse_matrix_y1.npz").toarray().reshape((-1, 1))
y_test = sparse.load_npz("/content/sparse_matrix_y2.npz").toarray().reshape((-1, 1))

y_train = torch.tensor(y_train, dtype=torch.int8)
y_val = torch.tensor(y_val, dtype=torch.int8)
y_test = torch.tensor(y_test, dtype=torch.int8)


print("Shapes:")
print("x_train:", X_train.shape)
print("x_val:", X_val.shape)
print("x_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_val:", y_val.shape)
print("y_test:", y_test.shape)

Shapes:
x_train: torch.Size([28683, 10000])
x_val: torch.Size([9562, 10000])
x_test: torch.Size([9562, 10000])
y_train: torch.Size([28683, 1])
y_val: torch.Size([9562, 1])
y_test: torch.Size([9562, 1])


In [8]:
# number of benigns and malicious sample in training dataset
n_ben = (y_train.squeeze()== 0).sum().item()
n_mal = (y_train.squeeze()== 1).sum().item()
print('the proportion of malwares : ', n_mal/(n_mal+n_ben))

# Combine features and labels into datasets
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

# Define the DataLoader for training, validation, and test sets
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size,shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Clear unnecessary variables
del train_dataset, val_dataset, test_dataset, y_train, y_val, y_test


the proportion of malwares :  0.11386535578565701


In [9]:
model_DNN = MalwareDetectionModel().to(device)
# Load model parameters
model_DNN.load_state_dict(torch.load('model_DNN_drebin_best.pth', map_location=torch.device(device)))

<All keys matched successfully>

In [10]:
# Create an instance of your model
model_AT_rFGSM = MalwareDetectionModel().to(device)

# Load model parameters
model_AT_rFGSM.load_state_dict(torch.load('model_AT_rFGSM.pth', map_location=torch.device(device)))

<All keys matched successfully>

In [11]:
# Create an instance of your model
model_AT_rFGSM_weightedLoss = MalwareDetectionModel().to(device)

# Load model parameters
model_AT_rFGSM_weightedLoss.load_state_dict(torch.load('model_AT_rFGSM_weightedLoss.pth', map_location=torch.device(device)))

<All keys matched successfully>

In [12]:
from sklearn.cluster import KMeans

# Function to initialize centers and sigmas separately for benign and malware samples using KMeans clustering
def initialize_centers_sigmas_separate(data_loader, num_centers_per_class, removal_array = removal_array, non_removal_features = False):
    centers_benign = []
    centers_malware = []
    non_removal_mask = torch.logical_not(removal_array).to('cpu')

    # Collect data into a single tensor
    all_data = torch.cat([batch for batch, _ in data_loader], dim=0)
    if non_removal_features:
        all_data = all_data[:,non_removal_mask]

    all_data = all_data.numpy()
    # Collect labels into a single tensor
    all_labels = torch.cat([labels for _, labels in data_loader], dim=0)
    all_labels = all_labels.numpy()

    # Separate benign and malware samples
    benign_indices = np.where(all_labels == 0)[0]
    malware_indices = np.where(all_labels == 1)[0]

    # Clustering for benign samples
    benigns = all_data[benign_indices]
    if non_removal_features:
        subset_benigns = benigns
    else:
        subset_benigns = benigns[:20000]

    kmeans_benign = KMeans(n_clusters=num_centers_per_class, init='k-means++', n_init='auto')
    kmeans_benign.fit(subset_benigns)
    centers_selected_benign = kmeans_benign.cluster_centers_

    # Clustering for malware samples
    kmeans_malware = KMeans(n_clusters=num_centers_per_class, init='k-means++', n_init='auto')
    kmeans_malware.fit(all_data[malware_indices])
    centers_selected_malware = kmeans_malware.cluster_centers_

    # Combine selected centers
    all_centers = np.concatenate([centers_selected_benign, centers_selected_malware], axis=0)

    return all_centers

In [ ]:
all_centers = initialize_centers_sigmas_separate(train_loader, 500, removal_array = removal_array, non_removal_features = False)

In [ ]:
all_centers.shape

In [ ]:
# Save the array to a file
np.save('all_centers.npy', all_centers)

In [ ]:
# Replace negative values with zero
all_centers = np.maximum(all_centers, 0)
all_centers[0][:30]

In [ ]:
all_centers_non_removal_features, sigma_non_removal_features = initialize_centers_sigmas_separate(train_loader, 500, removal_array = removal_array, non_removal_features = True)

In [ ]:
print(all_centers_non_removal_features.shape)

(1000, 1144)
3.967844595954441


In [ ]:
all_centers_non_removal_features[0][:20]

array([ 4.76190476e-02, -5.55111512e-17,  4.76190476e-02, -4.16333634e-17,
        4.76190476e-02,  0.00000000e+00, -1.38777878e-17, -6.93889390e-18,
        9.52380952e-02, -5.55111512e-17,  4.76190476e-02, -2.77555756e-17,
       -1.38777878e-17,  8.09523810e-01, -6.93889390e-18,  0.00000000e+00,
        0.00000000e+00,  2.08166817e-17,  1.42857143e-01,  1.04083409e-17])

In [ ]:
# Save the array to a file
np.save('all_centers_non_removal_features.npy', all_centers_non_removal_features)

In [ ]:
# Load the array from the file
all_centers = np.load('all_centers.npy')

In [ ]:
# Move all_centers back to GPU
all_centers = torch.tensor(all_centers, device=device)

In [ ]:
all_centers_non_removal_features = torch.tensor(all_centers_non_removal_features, device=device)

<ipython-input-20-b585ae409818>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_centers_non_removal_features = torch.tensor(all_centers_non_removal_features, device=device)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from sklearn.cluster import KMeans

# Define the RBF model
class RBFModel(nn.Module):
    def __init__(self, hidden_dim, output_dim, init_centers, init_sigmas, kernel):
        super(RBFModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.centers = nn.Parameter(torch.Tensor(init_centers))
        self.sigmas = nn.Parameter(torch.Tensor(init_sigmas))
        self.kernel = kernel
        # Linear layer for output
        self.linear = nn.Linear(hidden_dim, output_dim)

    def gaussian(self, x, c, sigma):
        return torch.exp(-torch.sum((x[:, None, :] - c) ** 2, dim=-1) / (2 * sigma ** 2))

    def laplacian(self, x, c, sigma):
        return torch.exp(-torch.sum(torch.abs(x[:, None, :] - c) , dim=-1) / sigma)

    def forward(self, x):
      if self.kernel == 'gaussian':
        radial_out = self.gaussian(x, self.centers, self.sigmas)
      elif self.kernel == 'laplacian':
        radial_out = self.laplacian(x, self.centers, self.sigmas)
      else:
        raise ValueError("Invalid kernel type. Choose 'gaussian' or 'laplacian'.")

      output = self.linear(radial_out.to(torch.float32))
      return output

# Function to evaluate the model on a dataset
def evaluate_model(model, data_loader, criterion, removal_array = removal_array, non_removal_features = False):
    model.eval()
    total_loss = 0.0
    correct_predictions = 0
    non_removal_mask = torch.logical_not(removal_array).to('cpu')

    with torch.no_grad():
        for batch_x, batch_y in data_loader:
            if non_removal_features:
                batch_x, batch_y = batch_x[:,non_removal_mask].to(torch.float32).to(device), batch_y.to(device)  # Move data to GPU
            else:
                batch_x, batch_y = batch_x.to(torch.float32).to(device), batch_y.to(device)  # Move data to GPU
            output = model(batch_x)
            loss = criterion(output, batch_y.squeeze().to(torch.long))
            total_loss += loss.item()
            _, predicted = torch.max(output, 1)
            correct_predictions += (predicted == batch_y.squeeze()).sum().item()

    avg_loss = total_loss / len(data_loader)
    accuracy = correct_predictions / len(data_loader.dataset)

    return avg_loss, accuracy


In [ ]:
# Run this to release GPU memory
torch.cuda.empty_cache()

In [ ]:
non_removal_features = False
non_removal_mask = torch.logical_not(removal_array).to('cpu')
sigma = 20.

model = RBFModel(1000, 2, all_centers, [sigma])
model = model.to(device)

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)  # Decrease lr by a factor of 0.95 every 1 epoch

# Training loop
num_epochs = 30
best_val_accuracy = 0.0
best_model_state_dict = None

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    correct_predictions = 0

    # Print the current learning rate
    current_lr = optimizer.param_groups[0]['lr']
    print('************************************')
    print(f'Current Learning Rate: {current_lr}')

    for i, (batch_x, batch_y) in enumerate(train_loader):
        # Remove features based on removal_array
        if non_removal_features:
            batch_x = batch_x[:, non_removal_mask].to(torch.float32).to(device)  # Select features
        else:
            batch_x = batch_x.to(torch.float32).to(device)  # Move all features to GPU
        batch_y = batch_y.to(device)  # Move labels to GPU

        optimizer.zero_grad()
        output = model(batch_x)
        loss = criterion(output, batch_y.squeeze().to(torch.long))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(output, 1)
        correct_predictions += (predicted == batch_y.squeeze()).sum().item()

        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item()}')

    avg_loss = total_loss / len(train_loader)
    accuracy = correct_predictions / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_loss}, Train Accuracy: {accuracy}')

    # Step the scheduler
    scheduler.step()

    # Evaluate on validation set
    val_loss, val_accuracy = evaluate_model(model, val_loader, criterion, removal_array = removal_array, non_removal_features=non_removal_features)
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

    # Check if the current model has the best validation accuracy
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        print(f"New best validation accuracy: {best_val_accuracy}")
        # Save the current best model state dict
        best_model_state_dict = model.state_dict()

# Save the best model
if best_model_state_dict is not None:
    torch.save(best_model_state_dict, 'best_model3.pth')
    print("Best model saved with validation accuracy:", best_val_accuracy)
else:
    print("No improvement in validation accuracy.")


In [ ]:
non_removal_features = True
non_removal_mask = torch.logical_not(removal_array).to('cpu')

model = RBFModel(1000, 2, all_centers_non_removal_features, [sigma_non_removal_features])
model = model.to(device)

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)  # Decrease lr by a factor of 0.95 every 1 epoch

# Training loop
num_epochs = 30
best_val_accuracy = 0.0
best_model_state_dict = None

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    correct_predictions = 0

    # Print the current learning rate
    current_lr = optimizer.param_groups[0]['lr']
    print('************************************')
    print(f'Current Learning Rate: {current_lr}')

    for i, (batch_x, batch_y) in enumerate(train_loader):
        # Remove features based on removal_array
        if non_removal_features:
            batch_x = batch_x[:, non_removal_mask].to(torch.float32).to(device)  # Select features
        else:
            batch_x = batch_x.to(torch.float32).to(device)  # Move all features to GPU
        batch_y = batch_y.to(device)  # Move labels to GPU

        optimizer.zero_grad()
        output = model(batch_x)
        loss = criterion(output, batch_y.squeeze().to(torch.long))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(output, 1)
        correct_predictions += (predicted == batch_y.squeeze()).sum().item()

        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item()}')

    avg_loss = total_loss / len(train_loader)
    accuracy = correct_predictions / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_loss}, Train Accuracy: {accuracy}')

    # Step the scheduler
    scheduler.step()

    # Evaluate on validation set
    val_loss, val_accuracy = evaluate_model(model, val_loader, criterion, removal_array = removal_array, non_removal_features=non_removal_features)
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

    # Check if the current model has the best validation accuracy
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        print(f"New best validation accuracy: {best_val_accuracy}")
        # Save the current best model state dict
        best_model_state_dict = model.state_dict()

# Save the best model
if best_model_state_dict is not None:
    torch.save(best_model_state_dict, 'best_model3.pth')
    print("Best model saved with validation accuracy:", best_val_accuracy)
else:
    print("No improvement in validation accuracy.")


************************************
Current Learning Rate: 0.0005
Epoch [1/30], Step [10/113], Loss: 0.07187538594007492
Epoch [1/30], Step [20/113], Loss: 0.09854941815137863
Epoch [1/30], Step [30/113], Loss: 0.04180261120200157
Epoch [1/30], Step [40/113], Loss: 0.058634303510189056
Epoch [1/30], Step [50/113], Loss: 0.09666981548070908
Epoch [1/30], Step [60/113], Loss: 0.07460201531648636
Epoch [1/30], Step [70/113], Loss: 0.13678571581840515
Epoch [1/30], Step [80/113], Loss: 0.10045940428972244
Epoch [1/30], Step [90/113], Loss: 0.1356290876865387
Epoch [1/30], Step [100/113], Loss: 0.13436344265937805
Epoch [1/30], Step [110/113], Loss: 0.08568780869245529
Epoch [1/30], Train Loss: 0.07825522694275179, Train Accuracy: 0.9724226893979012
Epoch [1/30], Validation Loss: 0.07745550651299327, Validation Accuracy: 0.9741685839782472
New best validation accuracy: 0.9741685839782472
************************************
Current Learning Rate: 0.000475
Epoch [2/30], Step [10/113], Loss:

In [ ]:
# After training, you can extract the trained parameters
trained_centers = model.centers.detach().cpu().numpy()
trained_sigmas = model.sigmas.detach().cpu().numpy()

# Print or use the trained parameters
print("Trained Centers:", trained_centers)
print("Trained Sigmas:", trained_sigmas)

Trained Centers: [[-6.72877218e-02  5.28677201e-02  1.48266270e-01 ...  2.69777879e-06
   3.73975518e-06 -9.17385143e-04]
 [-2.14057201e-01  1.86571142e-02 -9.47955544e-01 ... -2.88002616e-06
  -1.12124075e-06 -9.37939458e-04]
 [-2.22309262e-01 -1.64610102e-01 -7.93011726e-01 ... -3.61319814e-06
  -2.56563590e-06 -8.19786334e-04]
 ...
 [ 7.84233617e-02 -1.96447672e-01  1.18477684e+00 ...  2.95412757e-05
   2.15170042e-05  1.09293086e-03]
 [ 3.41224334e-01  2.15215011e-01  1.06755409e+00 ...  3.61441717e-05
   8.51299991e-05  1.59761195e-03]
 [ 7.56436409e-01 -3.89683388e-02 -1.01914606e+00 ... -3.12372701e-06
  -2.06403810e-06 -9.01615991e-04]]
Trained Sigmas: [3.272349]


In [ ]:
trained_centers.shape

(1000, 1144)

In [ ]:
trained_centers[0][:100]

array([-0.06728772,  0.05286772,  0.14826627,  0.10858831,  0.73251667,
       -0.31598319,  0.84853289, -0.3052243 ,  0.4397249 ,  0.2005334 ,
        0.91057174, -0.10050466, -0.03121453,  1.00701555,  0.27813542,
        0.08260053, -0.27219494,  0.14633815,  0.50917671, -0.23590404,
        0.17625967, -0.07525321,  0.33876415, -0.23252008,  0.39667067,
       -0.52118714,  0.28684472,  0.12878577, -0.45861407, -0.20414072,
       -0.19059005, -0.14260681, -0.19978095, -0.38665129, -0.0157628 ,
       -0.18960417, -0.18796049, -0.50119469, -0.18496574, -0.15641157,
       -0.14795656, -0.12750056, -0.02451814, -0.41335375, -0.29829602,
       -0.38242726,  0.11990683,  0.28861171,  0.02903654,  0.19225395,
       -0.16946435, -0.16863965, -0.13654089,  0.25108567, -0.23583435,
        0.55016197,  0.3419944 ,  0.46616686, -0.28680909,  0.30642635,
       -0.04406383, -0.30059522, -0.12552965, -0.1763888 , -0.18170115,
       -0.25665917,  0.07063957,  0.50430855, -0.18616898, -0.12

In [ ]:
# Re-instantiate the model
all_centers = torch.rand((1000, 10000))
sigma = 5.
model = RBFModel(1000, 2, all_centers, [sigma])

# Load the saved model state dictionary
model_state_dict = torch.load('/content/drive/MyDrive/best_model.pth')

# Load the model state dictionary into the model
model.load_state_dict(model_state_dict)

model = model.to(device)

# Ensure the model is in evaluation mode
model.eval()


RBFModel(
  (linear): Linear(in_features=1000, out_features=2, bias=True)
)

In [ ]:
# Re-instantiate the model
model_non_removal_features = RBFModel(1000, 2, all_centers_non_removal_features, [sigma_non_removal_features])

# Load the saved model state dictionary
model_state_dict = torch.load('best_model_non_removal_features3.pth')

# Load the model state dictionary into the model
model_non_removal_features.load_state_dict(model_state_dict)

model_non_removal_features = model_non_removal_features.to(device)

# Ensure the model is in evaluation mode
model_non_removal_features.eval()


RBFModel(
  (linear): Linear(in_features=1000, out_features=2, bias=True)
)

In [ ]:
def model_evaluation(model, test_loader, device):
    model.eval()  # Set the model to evaluation mode
    predictions = []
    true_labels = []

    with torch.no_grad():
        for X_batch, labels_batch in test_loader:
            X_batch, labels_batch = X_batch[:, non_removal_mask].to(torch.float32).to(device), labels_batch.to(device)
            outputs = model(X_batch)
            _, predicted = torch.topk(outputs, k=1)
            predictions.extend(predicted.tolist())
            true_labels.extend(labels_batch.tolist())

    # Convert predictions and true labels to numpy arrays
    predictions = np.array(predictions)
    true_labels = np.array(true_labels)

    # Calculate and print test accuracy
    accuracy = accuracy_score(true_labels, predictions)
    balanced_acc = balanced_accuracy_score(true_labels, predictions)
    print(f'Test Accuracy: {accuracy:.4f}')
    print(f'Test balanced Accuracy: {balanced_acc:.4f}')

    # Calculate and print precision, recall, and F1-score
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)

    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1-score: {f1:.4f}')

    # Calculate and print true positives, true negatives, false positives, and false negatives
    TP = ((predictions == 1) & (true_labels == 1)).sum()
    TN = ((predictions == 0) & (true_labels == 0)).sum()
    FP = ((predictions == 1) & (true_labels == 0)).sum()
    FN = ((predictions == 0) & (true_labels == 1)).sum()

    print(f'True Positives (TP): {TP}')
    print(f'True Negatives (TN): {TN}')
    print(f'False Positives (FP): {FP}')
    print(f'False Negatives (FN): {FN}')

    # Calculate and print False Negative Rate (FNR) and False Positive Rate (FPR)
    FNR = (FN / (FN + TP)) * 100
    FPR = (FP / (FP + TN)) * 100

    print(f'False Negative Rate (FNR): {FNR:.4f}')
    print(f'False Positive Rate (FPR): {FPR:.4f}')

In [ ]:
model_evaluation(model, test_loader, device)

Test Accuracy: 0.9855
Test balanced Accuracy: 0.9623
Precision: 0.9444
Recall: 0.9319
F1-score: 0.9381
True Positives (TP): 1053
True Negatives (TN): 8370
False Positives (FP): 62
False Negatives (FN): 77
False Negative Rate (FNR): 6.8142
False Positive Rate (FPR): 0.7353


In [ ]:
model_evaluation(model_non_removal_features, test_loader, device)

Test Accuracy: 0.9723
Test balanced Accuracy: 0.9099
Precision: 0.9295
Recall: 0.8283
F1-score: 0.8760
True Positives (TP): 936
True Negatives (TN): 8361
False Positives (FP): 71
False Negatives (FN): 194
False Negative Rate (FNR): 17.1681
False Positive Rate (FPR): 0.8420


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
